<a href="https://colab.research.google.com/github/doudi25/BackProp-Ninja-with-binary-classification-task/blob/main/BackPropNinja_with_sigmoid_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.nn import functional as F


In [ ]:
w1 = torch.randn(4,5,requires_grad=True)
w2 = torch.randn(5,1,requires_grad=True)


In [ ]:
data = torch.randn(10,4)


In [ ]:
o1 = data @ w1
o2 = o1 @ w2
out = torch.sigmoid(o2)

In [ ]:
real_label = torch.cat([torch.ones(5),torch.zeros(5)])

In [ ]:
loss = torch.nn.BCELoss()
loss_fn = loss(out.view(-1),real_label)

In [ ]:
loss_fn.backward()

In [ ]:
loss2 = - (real_label * torch.log(out.view(-1)) + (1 - real_label) * torch.log(1 - out.view(-1)))


In [ ]:
loss_fn

tensor(1.2539, grad_fn=<BinaryCrossEntropyBackward0>)

In [ ]:
loss2.mean()

tensor(1.2539, grad_fn=<MeanBackward0>)

In [ ]:
pb = torch.zeros(10)
dpb = torch.ones(10)
# pb is the vector of the binary cross entropy loss , -log(y) for pb of 1 -log(1-y) for pb of 0
for i in range(10):
  if i < 5 :
    pb[i] = -torch.log(out[i])
  else :
    pb[i] = -torch.log(1-out[i])
dpb = dpb/10

In [ ]:
dy = torch.zeros(10)
for i in range(10):
  if i < 5 :
    dy[i] = dpb[i] * (-1 /out[i])
  else :
    dy[i] = dpb[i] * (1/(1-out[i]))

In [ ]:
dlogits = (out.view(-1) * (1 - out.view(-1)) )* dy

In [ ]:
dlogits

tensor([-0.0776, -0.0529, -0.0966, -0.0997, -0.0282,  0.0456,  0.0050,  0.0033,
         0.0017,  0.0002], grad_fn=<MulBackward0>)

In [ ]:
# apply directly the gradient on the logits and compare it to my backprop
dout = out
dout[range(5)] += -1
dout = dout/10

In [ ]:
dout

tensor([[-0.0776],
        [-0.0529],
        [-0.0966],
        [-0.0997],
        [-0.0282],
        [ 0.0456],
        [ 0.0050],
        [ 0.0033],
        [ 0.0017],
        [ 0.0002]], grad_fn=<DivBackward0>)

In [ ]:
dw2 = o1.T @ dlogits.view(-1,1)

In [ ]:
do1 = dlogits.view(-1,1) @ w2.T

In [ ]:
dw1 = data.T  @ do1

In [ ]:
torch.allclose(dw2,w2.grad)

True

In [ ]:
torch.allclose(dw1,w1.grad)

True